I have observed a tendency in my peers to write in a procedural, nested-loop-heavy, index-heavy style reminiscent of C, Fortran, or Matlab. Matlab was heavily influenced by Fortran (citation needed?) and both C and Fortran are fairly low-level languages in which this style is often necessary. Python, however, is a high-level scripting language and has many alternatives to this style. Indices should be avoided, and can be avoided in most cases. A weaker, but similar statement can be made for loops, and in particular nested loops.

Often I see people code in the procedural paradigm. Presumably this is because they are primed to approach mathematical algorithms in this way, either by algorithm pseudo code in a textbook, or because they are converting code from another language; or perhaps they are unaware of the alternatives. Functional programming (my preference) and object oriented programming have significant benefits, and ought to be considered when appropriate.

# Example: Element-wise Squaring

The C/Fortran approach is to iterate over indices. This should be avoided in Python as it is slow and less readable than the alternatives. The following code demonstrates three ways to perform element-wise squaring of a Python list, from least Pythonic to most Pythonic. Not only is idomatic Python more readable, it is faster.

In [47]:
# generate some data
xs = list(range(10))

# C-style looping over indices
# This is unpythonic and strongly discouraged.
def element_wise_square_C_like(xs):
    new_xs = []
    for i in range(len(xs)):
        new_xs.append( xs[i]**2 )
    return new_xs
print(element_wise_square_C_like(xs))

# More Pythonic: iterating over elements rather than indices
def element_wise_square_iterate_over_element(xs):
    new_xs = []
    for x in xs:
        new_xs.append( x**2 )
    return new_xs
    
print(element_wise_square_iterate_over_element(xs))

# List comprehensions are the most Pythonic
def element_wise_square_comprehension(xs):
    return [x**2 for x in xs]
print(element_wise_square_comprehension(xs))

# using the map function is a more funtional approach
#      in particular, it makes parallelzation trivial
def square(x):
    return x**2
def element_wise_square_map(xs):
    return list(map(square, xs))
print(element_wise_square_map(xs))

n = 10**5
xs = list(range(n))
print('\nTimings:\n')
print('indexing\n\t', end='')
%timeit element_wise_square_C_like(xs)
print('iterating over elements\n\t', end='')
%timeit element_wise_square_iterate_over_element(xs)
print('list comprehension\n\t', end='')
%timeit element_wise_square_comprehension(xs)
print('map function\n\t', end='')
%timeit element_wise_square_map(xs)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

Timings:

indexing
	34.4 ms ± 186 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
iterating over elements
	31.2 ms ± 172 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
list comprehension
	27.9 ms ± 105 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
map function
	31.3 ms ± 232 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Numpy
For numerical computation, we often use the package `NumPy`, which is said to *vectorize* operations. Essentially, this means we call a compiled function to perform the looping, which executes much faster than the equivalent Python. **Be warrned:** though Numpy can be much faster, incorrect usage will dramatically slow down your code.

In [50]:
# Numpy must be imported. This is the common convention.
import numpy as np
#from numpy import *
'''
Avoid importing numpy in this way. Though it may be annoying to type 
the prefix "np.", you may want to use similarly named functions from
other namespaces such as "math." or "sym.".
'''
# create the data
xs_array = np.array(range(10))

# vectorized - very fast
def element_wise_square_numpy_vectorized(xs_array):
    return xs_array**2
print(element_wise_square_iterate_over_element(xs_array))

# pre-allocating and looping over indices - slow
def element_wise_square_numpy_preallocated_loop(xs_array):
    n = len(xs_array)
    new_array = np.empty(n, dtype=int)
    for i in range(n):
        new_array[i] = xs_array[i]**2
    return new_array
print(element_wise_square_numpy_preallocated_loop(xs_array))

# using np.append - Incredibly slow!
def element_wise_square_numpy_append(xs_array):
    n = len(xs_array)
    new_array = np.array([], dtype=int)
    for i in range(n):
        new_array = np.append(new_array, xs_array[i]**2)
    return new_array
print(element_wise_square_numpy_append(xs_array))

n = 10**5
xs_array = np.array(range(n))
print('\nTimings:\n')
print('vectorized\n\t', end='')
%timeit element_wise_square_numpy_vectorized(xs_array)
print('preallocate then loop\n\t', end='')
%timeit element_wise_square_numpy_preallocated_loop(xs_array)
print('appending\n\t', end='')
%timeit element_wise_square_numpy_append(xs_array)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
[ 0  1  4  9 16 25 36 49 64 81]
[ 0  1  4  9 16 25 36 49 64 81]

Timings:

vectorized
	72.7 µs ± 1.91 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
preallocate then loop
	33.9 ms ± 641 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
appending
	3.41 s ± 29.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Tools for Avoiding Indices

### The enumerate function
Some algorithms *require* indices as part of the computation, rather than simply used as an index. The following example evaluates a polynomial given an input value and a list of coefficients. In such cases we can use the `enumerate` function to create Pythonic code.

In [35]:
coeffs = [1, 4, 3, 2] # representing the polynomial f(x) = 1 + 4x + 3x^2 + 2x^3
x = 4

# The C/Fortran style - avoid this
def poly_eval_indices(x, coeffs):
    val = 0
    for i in range(len(coeffs)):
        val += coeffs[i]*x**i
    return val
print(poly_eval_indices(x, coeffs))

# It is more pythonic to use the enumerate function and avoid indexing
def poly_eval_enumerate(x, coeffs):
    val = 0
    for i, c in enumerate(coeffs):
        val += c*x**i
    return val
print(poly_eval_enumerate(x, coeffs))

# Even better, we can use a list comprehension
def poly_eval_list_comprehension(x, coeffs):
    return sum([c*x**i for i,c in enumerate(coeffs)])
print(poly_eval_list_comprehension(x, coeffs))

# Generator comprehensions will save memory
def poly_eval_generator_comprehension(x, coeffs):
    return sum(c*x**i for i,c in enumerate(coeffs))
print(poly_eval_generator_comprehension(x, coeffs))

193
193
193
193


### The Zip Function
Often we need coresponding elements from similarly sized lists. The `zip` function allows us to avoid indices.

In [12]:
xs = list(range(10))
ys = list(range(10, 20))

# C/Fortran style - avoid this
def pairwise_add_indexing(xs, ys):
    ret = []
    for i in range(len(xs)):
        ret.append(xs[i] + ys[i])
    return ret
print(pairwise_add_indexing(xs, ys))

# using zip
def pairwise_add_zip(xs, ys):
    ret = []
    for x, y in zip(xs, ys):
        ret.append(x+y)
    return ret
print(pairwise_add_zip(xs, ys))

# using zip and a comprehension - very pythonic
def pairwise_add_comprehension(xs,ys):
    return [x+y for x,y in zip(xs,ys)]
print(pairwise_add_comprehension(xs, ys))

# using starmap - a functional version, perhaps less pythonic
from itertools import starmap
from operator import add
def pairwise_add_star_map(xs,ys):
    return list(starmap(add, zip(xs, ys)))
print(pairwise_add_comprehension(xs, ys))

[10, 12, 14, 16, 18, 20, 22, 24, 26, 28]
[10, 12, 14, 16, 18, 20, 22, 24, 26, 28]
[10, 12, 14, 16, 18, 20, 22, 24, 26, 28]


# Collapsing Nested Loops
Python has relevant whitespace. When using many nested loops, the indentation can become significant and cause the code to be less readable. For many applications, this can be improved by using the `iterools.product` function, which iterates over a cartesian product. The following code shows a few ways to find the maximum of a function on a discrete domain.

In [62]:
def foo(a,b,c,d,e):
    return (a-b)**(e-d) % c
a_domain = [1, 5, 10, 18]
b_domain = [-2, 4, 5]
c_domain = [13, 15, 19]
d_domain = [1, 2, 3, 4]
e_domain = [5, 6, 7, 8]

# find the max with nested loops
my_max = float('-inf')
for a in a_domain:
    for b in b_domain:
        for c in c_domain:
            for d in d_domain:
                for e in e_domain:
                    val = foo(a,b,c,d,e)
                    if val > my_max:
                        my_max = val
print(my_max)

# using the product function
from itertools import product
my_max = float('-inf')
for a, b, c, d, e in product(a_domain, b_domain, c_domain, d_domain, e_domain):
    val = foo(a,b,c,d,e)
    if val > my_max:
        my_max = val
print(my_max)

# using a generator comprehension (more memory efficient than a list comprehension)
my_max = max(foo(a,b,c,d,e) for a, b, c, d, e in product(a_domain, b_domain, c_domain, d_domain, e_domain))
print(my_max)

# a functional approach
my_max = max( starmap(foo, product(a_domain, b_domain, c_domain, d_domain, e_domain) ) )
print(my_max)

18
18
18
18
